In [1]:
%run ../script/webnlg.py

import pandas as pd
import spacy
from textacy import similarity

pd.set_option('max_colwidth', 1000)

nlp = spacy.load('en_core_web_lg')

# Data Alignment

In [11]:
def get_span(doc, node):

    return doc[node.left_edge.i: node.right_edge.i + 1]

def get_left_span(doc, node):

    return doc[node.left_edge.i: node.i + 1]

def get_right_span(doc, node):

    return doc[node.i: node.right_edge.i + 1]

def as_span(doc, node):

    return doc[node.i: node.i + 1]

class RootDataAlignmentModel:
    
    def __init__(self, similarity_metric):
        
        self.similarity_metric = similarity_metric
        

    def get_distances(self, doc, data):

        distances, spans = [], []

        # dependency trees' roots
        roots = [token for token in doc if token.head == token]

        # BIAS: parts of the tree
        # breadth-first
        for root in roots:

            # root subtree
            # BIAS: parts of the tree
            root_span = get_span(doc, root)
            # root left subtree
            root_left_span = get_left_span(doc, root)
            # root right subtree
            root_right_span = get_right_span(doc, root)
            # root node
            only_root = as_span(doc, root)

            # test agains the node and its subtree
            for span in set((only_root, root_span, root_left_span, root_right_span)):

                spans.append(span)

                distances_span = []

                # for each structured data, calculate similarity
                for d in data.values():

                    distances_span.append(self.similarity_metric(d, span.text))

                distances.append(distances_span)
            
            # add children
            roots.extend(root.lefts)
            roots.extend(root.rights)

        return pd.DataFrame(distances, index=spans, columns=data.keys())

# Template Extraction

In [27]:
# tests if two spans overlaps
# TODO: move to spacy_util
def overlaps(span1, span2):

    return max(0, min(span1.end_char, span2.end_char) - max(span1.start_char, span2.start_char)) > 0


class TemplateExtractionModel:
    
    def __init__(self, data_alignment_model):
    
        self.data_alignment_model = data_alignment_model
        

    def extract_template(self, text, data):

        # spacy model
        doc = nlp(text)

        # use data alignment model to get similarity matrix
        df = self.data_alignment_model.get_distances(doc, data)

        # breaks text into char array
        text_char = list(text)

        # subject extraction
        # BIAS: subject wins priority over distances tie
        m_subject_span = df.m_subject.nlargest(1).index.values[0]

        # replaces subject text with m_subject placeholder
        text_char[m_subject_span.start_char: m_subject_span.end_char] = '{m_subject}'

        # object extraction
        # search for the best span for object, different from the subject one
        for span in df.m_object.sort_values(ascending=False).index.values:

            # tests if the current span doesn't overlaps the subject one
            if overlaps(span, m_subject_span):

                continue

            # tests if the object occurs after the subject >
            #    if it is the case, you have to adjust the indexes accordingly
            if m_subject_span.start_char > span.end_char:

                base = 0
            else:
                # adjustes the indexes
                # length of the extracted subject text
                len_subject_text = m_subject_span.end_char - m_subject_span.start_char
                # length of the placeholder minus len_subject_text
                base = len('{m_subject}') - len_subject_text

            # replaces object text with m_object placeholder
            text_char[base + span.start_char: base + span.end_char] = '{m_object}'

            break

        # build template using the char array
        return ''.join(text_char)

# WebNLG

In [14]:
train = WebNLGCorpus.load('train')

# BIAS: use only 1 tripleset size dataset
train_1 = train.subset(ntriples=1)

In [21]:
# sample an entry
e = train_1.sample(idx='0_210')

# data alignment model with token_sort_ratio similarity metric
# PARAM/BIAS: similarity_metric 
# TODO: train changing similarity_metric
da = RootDataAlignmentModel(similarity.token_sort_ratio)
# template extraction model
te = TemplateExtractionModel(da)
    
# uses the first reference text
# it can have more than 1 reference text
first_lexicalization = e.ldf.ltext.values.tolist()[0]
# uses the first triple
first_triple = e.mdf[['m_subject', 'm_object']].to_dict(orient='records')[0]

first_lexicalization, first_triple

('The Faroese language is spoken in Denmark.',
 {'m_subject': 'Denmark', 'm_object': 'Faroese_language'})

In [22]:
# extracts the template
template = te.extract_template(first_lexicalization, first_triple)
template

'{m_object} is spoken in {m_subject}.'

In [25]:
template.format(**{'m_subject': 'Brazil', 'm_object': 'Portuguese'})

'Portuguese is spoken in Brazil.'

In [29]:
# TODO: remove lexicalized information not present in data
text = 'Eleanor Rigby picks up the rice in the church'
data = {'m_subject': 'Eleanor Rigby', 
        'm_object': 'rice'}

da = RootDataAlignmentModel(similarity.token_sort_ratio)
te = TemplateExtractionModel(da)

template = te.extract_template(text, data)

print(template)

template.format(**{'m_subject': 'Abelardo Vieira Mota', 'm_object': 'car'})

{m_subject} picks up the {m_object} in the church


'Abelardo Vieira Mota picks up the car in the church'

# Train

In [31]:
%%time

from collections import Counter, defaultdict

da = RootDataAlignmentModel(similarity.token_sort_ratio)
te = TemplateExtractionModel(da)

template_db = defaultdict(list)

#! BIAS: using only train_1 sentences
# for each sentence, extracts template
for entry in train_1:
    
    for text in entry.ldf.ltext.tolist():
        # to dictionary of s, o; [0] because to_dict returns a list of dicts(and, in this case, there
        #    will be only one element)
        data = entry.mdf[['m_subject', 'm_object']].to_dict(orient='records')[0]
        predicate = entry.mdf.m_predicate.values[0]

        template = te.extract_template(text, data)

        # add to db
        template_db[predicate].append((entry.edf.idx.values[0], template))
    
# most frequent template
for k, templates in template_db.items():
    
    template_db[k] = Counter([v[1] for v in templates]).most_common(1)[0][0]
    
predicates_in_db = list(template_db.keys())

CPU times: user 3min 25s, sys: 9.26 s, total: 3min 34s
Wall time: 59.7 s


In [32]:
len(template_db)

227

In [33]:
template_db['language']

'{m_object} is spoken in {m_subject}.'

## Do I have one predicate for each predicate in test set?

In [19]:
test = WebNLGCorpus.load('test_no_lex')

In [21]:
predicates_in_test = set(test.mdf.m_predicate.tolist())
predicates_w_template = template_db.keys()

len("There are {} predicates in test which don't have a template".format(len(predicates_in_test ^ predicates_w_template)))

60

# So, let them fall back to the nearest predicate and then to baseline model

In [164]:
import logging

logger = logging.Logger('TemplateBasedModel')

unwanted_separators = re.compile(r'(\||_)')
unwanted_multiple_empty = re.compile(r'\s+')

def preprocess_triple(s):
    
    sep_changed = unwanted_separators.sub(' ', s)
    mult_empty_removed = unwanted_multiple_empty.sub(' ', sep_changed)
    
    return mult_empty_removed.replace('"', '')

def get_nearest_predicate(predicate):
    
    distances = [(in_db, similarity.jaro_winkler(predicate, in_db)) for in_db in predicates_in_db]
    
    return min(distances, key=lambda v: v[1])


def generate_sentences(entry):
    
    texts = []
    
    for i, triple in entry.mdf.iterrows():

        m_predicate = triple.m_predicate

        if m_predicate in template_db:
            
            template = template_db[m_predicate]

            preprocessed_triple = triple[['m_subject', 'm_object']].apply(preprocess_triple)

            text = template.format(**preprocessed_triple.to_dict())

        else:
            
            nearest_predicate, similarity = get_nearest_predicate(m_predicate)
            
            if similarity > .4:
                
                logger.warning("Fallback nearest predicate for predicate %s", m_predicate)
                
                template = template_db[nearest_predicate]

                preprocessed_triple = triple[['m_subject', 'm_object']].apply(preprocess_triple)

                text = template.format(**preprocessed_triple.to_dict())
                
            else:
                
               # logger.warning("Fallback baseline for predicate %s", m_predicate)

                text = preprocess_triple(triple.mtext)

        texts.append(text)
    
    return texts

In [165]:
test_sample = test.sample(idx='0_419')

test_sample.mdf

,idx,mtext,m_subject,m_predicate,m_object
639,0_419,"Acta_Palaeontologica_Polonica | ISSN_number | ""0567-7920""",Acta_Palaeontologica_Polonica,ISSN_number,"""0567-7920"""
640,0_419,Acta_Palaeontologica_Polonica | LCCN_number | 60040714,Acta_Palaeontologica_Polonica,LCCN_number,60040714
641,0_419,"Acta_Palaeontologica_Polonica | abbreviation | ""Acta Palaeontol. Pol.""",Acta_Palaeontologica_Polonica,abbreviation,"""Acta Palaeontol. Pol."""


In [166]:
logger.setLevel(logging.WARN)
generate_sentences(test_sample)

['The ISSN number of Acta Palaeontologica Polonica is 0567-7920.',
 'The LCCN number of Acta Palaeontologica Polonica is 60040714.',
 'Acta Palaeontologica Polonica is abbreviated Acta Palaeontol. Pol.. Pol.']

# Generating texts for test set

In [167]:
import codecs

logger.setLevel(logging.WARN)

with codecs.open('models/template_based/output.txt', 'w', 'utf-8') as f:
    
    for entry in test:
        
        entry_sentences = generate_sentences(entry)
        
        entry_text = ' '.join(entry_sentences)
        
        f.write(entry_text)
        f.write('\n')

In [168]:
!head -100 models/template_based/output.txt | tail -10

English language is spoken in Castle (novel).
Eric Flint was born in Burbank, California.
Farrar, Straus and Giroux is the parent company of the Macmillan Publishers Press.
One of John Cowper Powys notable works is Oliver A Glastonbury Romance.
Soho Press is located in United States.
Faber and Faber is the publisher of The Secret Scripture.
Asian Americans are an ethnic group in the United United States.
English language is spoken in United States.
Weymouth Sands by 1634: The A Glastonbury Romance Affair.
Rolando Maran manages A.C. Chievo Verona.


In [169]:
!python ../evaluation/webnlg2017/webnlg-automatic-evaluation-v2/evaluation_v2.py --team_name template_based --team_filepath models/template_based/output.txt --outdir models/template_based/

Files creating finished for:  template_based


In [170]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl -lc ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference0.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference1.lex ../evaluation/webnlg2017/webnlg-automatic-evaluation/references/gold-all-cat-reference2.lex < models/template_based/template_based_all-cat.txt

BLEU = 37.01, 65.0/44.8/30.7/21.0 (BP=1.000, ratio=1.156, hyp_len=52216, ref_len=45189)
